In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
231,AFG,Asia,Afghanistan,2020-08-18,37599.0,3.0,77.857,1375.0,0.0,9.000,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
394,ALB,Europe,Albania,2020-08-18,7499.0,119.0,137.571,230.0,2.0,4.286,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
626,DZA,Africa,Algeria,2020-08-18,39025.0,442.0,473.286,1379.0,9.0,9.571,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-17,France,221088.0,0.0,30436.0,0.0,0.0,0.0
0,2020-08-17,China,89445.0,0.0,4710.0,0.0,0.0,0.0
0,2020-08-17,Italy,254577.0,0.0,35552.0,0.0,0.0,0.0
0,2020-08-17,Spain,343292.0,0.0,28625.0,0.0,0.0,0.0
0,2020-08-17,United States,5496403.0,0.0,171242.0,0.0,0.0,0.0
0,2020-08-17,World,22196232.0,0.0,778547.0,0.0,0.0,0.0
0,2020-08-17,United Kingdom,319688.0,0.0,41427.0,0.0,0.0,0.0
0,2020-08-17,Germany,224672.0,0.0,9239.0,0.0,0.0,0.0
0,2020-08-17,Iran,347236.0,0.0,19891.0,0.0,0.0,0.0
0,2020-08-17,Turkey,250411.0,0.0,5993.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-18,France,221088.0,219029.0,30436.0,30429.0,2059.0,7.0
0,2020-08-18,China,89445.0,89441.0,4710.0,4703.0,4.0,7.0
0,2020-08-18,Italy,254577.0,254235.0,35552.0,35400.0,342.0,152.0
0,2020-08-17,Spain,343292.0,359082.0,28625.0,28646.0,-15790.0,-21.0
0,2020-08-18,United States,5496403.0,5438325.0,171242.0,170497.0,58078.0,745.0
0,2020-08-18,World,22196232.0,21896155.0,778547.0,774539.0,300077.0,4008.0
0,2020-08-18,United Kingdom,319688.0,319197.0,41427.0,41369.0,491.0,58.0
0,2020-08-18,Germany,224672.0,224014.0,9239.0,9232.0,658.0,7.0
0,2020-08-18,Iran,347236.0,345450.0,19891.0,19804.0,1786.0,87.0
0,2020-08-18,Turkey,250411.0,250542.0,5993.0,5996.0,-131.0,-3.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")